# Inpainting from [Deep Image Prior Paper](https://github.com/DmitryUlyanov/deep-image-prior) and [GitHub Repo](https://dmitryulyanov.github.io/deep_image_prior)

Set Runtime -> Change runtime type -? Under Hardware Accelerator select GPU in Google Colab

## Import libs

In [ ]:
"""
Set Runtime -> Change runtime type -> Under Hardware Accelerator select GPU in Google Colab 
"""
!git clone https://github.com/DmitryUlyanov/deep-image-prior
!mv deep-image-prior/* ./

Cloning into 'deep-image-prior'...
remote: Enumerating objects: 289, done.
remote: Total 289 (delta 0), reused 0 (delta 0), pack-reused 289
Receiving objects: 100% (289/289), 24.28 MiB | 25.22 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




---



Download our trainval/test split of PASCALVOC2012 [here](https://drive.google.com/file/d/1p2ZkFiNQR0WbE28x8ukLzCUBjUy9XXM4/view?usp=sharing). Upload it to your goolge drive then change the path in the code cell below to reflect where you saved original_data.zip in your google drive.

In [ ]:
!cp /content/drive/MyDrive/VLR_Project/data/PASCAL2012/original_data.zip /content/ && unzip /content/original_data.zip

Streaming output truncated to the last 5000 lines.
  inflating: original_data/JPEGImages/trainval/2008_008517.jpg  
  inflating: __MACOSX/original_data/JPEGImages/trainval/._2008_008517.jpg  
  inflating: original_data/JPEGImages/trainval/2008_001255.jpg  
  inflating: __MACOSX/original_data/JPEGImages/trainval/._2008_001255.jpg  
  inflating: original_data/JPEGImages/trainval/2009_000523.jpg  
  inflating: __MACOSX/original_data/JPEGImages/trainval/._2009_000523.jpg  
  inflating: original_data/JPEGImages/trainval/2008_001533.jpg  
  inflating: __MACOSX/original_data/JPEGImages/trainval/._2008_001533.jpg  
  inflating: original_data/JPEGImages/trainval/2012_000627.jpg  
  inflating: __MACOSX/original_data/JPEGImages/trainval/._2012_000627.jpg  
  inflating: original_data/JPEGImages/trainval/2011_003693.jpg  
  inflating: __MACOSX/original_data/JPEGImages/trainval/._2011_003693.jpg  
  inflating: original_data/JPEGImages/trainval/2008_007142.jpg  
  inflating: __MACOSX/original_data/JP

In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import numpy as np
from models.resnet import ResNet
from models.unet import UNet
from models.skip import skip
import torch
import torch.optim

from utils.inpainting_utils import *

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

PLOT = True
imsize = -1
dim_div_by = 64

## Set up images to augment

Download the file names of the images that we are going to augment from the original dataset [here](https://drive.google.com/file/d/18ijAncbO8jVh8xTV0XQi17uFYEk3LH7g/view?usp=sharing). These files were determined based on the files that were eligible for selective erasing in [this notebook](). Upload this file to the Colab File System.

In [ ]:
mask_list = []
with open("/content/names_augmented.txt") as files: 
  for file in files:
    mask_list.append(file.split("\n")[0])

## SETUP

Importing necessary libraries to genearte saliency map using EigenCAM from the [Pytorch Grad-CAM library](https://github.com/jacobgil/pytorch-grad-cam).

This is set up to specifically use Faster R-CNN with a ResNet-50 backbone rom Pytorch Vision Library.

In [ ]:
!git clone https://github.com/katelyn98/pytorch-grad-cam.git
!cd pytorch_grad_cam && pip install -r requirements.txt
!pip install requests --quiet
# !pip install grad-cam --quiet

import torchvision.transforms as transforms
import torchvision
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import glob
import os
import torch
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')  
import cv2
import numpy as np
import math
import requests
from pytorch_grad_cam import EigenCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, scale_cam_image
from pytorch_grad_cam.utils.model_targets import FasterRCNNBoxScoreTarget
from scipy.misc import face
from scipy.ndimage import zoom
from scipy.special import logsumexp
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

DEVICE = "cuda"

#prediction function for SSD model
coco_names = ['__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', \
              'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 
              'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 
              'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella',
              'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
              'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
              'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass', 'cup', 'fork',
              'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
              'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
              'potted plant', 'bed', 'N/A', 'dining table', 'N/A', 'N/A', 'toilet',
              'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
              'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book', 'clock', 'vase',
              'scissors', 'teddy bear', 'hair drier', 'toothbrush']


def predict(input_tensor, model, device, detection_threshold):
    outputs = model(input_tensor)
    pred_classes = [coco_names[i] for i in outputs[0]['labels'].cpu().numpy()]
    pred_labels = outputs[0]['labels'].cpu().numpy()
    pred_scores = outputs[0]['scores'].detach().cpu().numpy()
    pred_bboxes = outputs[0]['boxes'].detach().cpu().numpy()
    
    boxes, classes, labels, indices = [], [], [], []
    for index in range(len(pred_scores)):
        if pred_scores[index] >= detection_threshold:
            boxes.append(pred_bboxes[index].astype(np.int32))
            classes.append(pred_classes[index])
            labels.append(pred_labels[index])
            indices.append(index)
    boxes = np.int32(boxes)
    return boxes, classes, labels, indices

frcnn_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
frcnn_model.eval().cuda()

target_layers = [frcnn_model.backbone.body.layer4] #ssd
cam = EigenCAM(frcnn_model,
               target_layers, 
               use_cuda=torch.cuda.is_available())

Cloning into 'pytorch-grad-cam'...
remote: Enumerating objects: 796, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 796 (delta 39), reused 65 (delta 32), pack-reused 702
Receiving objects: 100% (796/796), 17.11 MiB | 13.15 MiB/s, done.
Resolving deltas: 100% (419/419), done.
/bin/bash: line 0: cd: pytorch_grad_cam: No such file or directory
     |████████████████████████████████| 4.5 MB 12.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

### Function to erase pixels based on a threshold - default erases top 2.5% salient pixels.

In [ ]:
def erase_black(attributions, img_orig, threshold = 97.5):
  mask = attributions <= np.percentile(attributions, threshold)
  im_mask = np.array(img_orig)
  im_mask[~mask] = 0
  mask_img = mask.astype(np.uint8)  #convert to an unsigned byte
  mask_img*=255
  return im_mask, mask_img

### Get the image mask by generating the saliency map.

In [ ]:
NET_TYPE = 'ResNet' # one of skip_depth4|skip_depth2|UNET|ResNet
transform = transforms.Compose([transforms.Resize((512, 512)), 
                                      transforms.ToTensor(),
                                      ])

def get_image_mask(img_path):
  img = Image.open(img_path)
  tensor = transform(img).unsqueeze(0)
  img = np.array(img)
  img = cv2.resize(img, (512, 512))
  img_arr = np.float32(img) / 255
  tensor = tensor.cuda()
  #SSD
  boxes, classes, labels, indices = predict(tensor, frcnn_model, DEVICE, 0.9)
  targets = [FasterRCNNBoxScoreTarget(labels=labels, bounding_boxes=boxes)]

  grayscale_cam = cam(tensor, targets=targets)
  grayscale_cam = grayscale_cam[0, :] 

  inpainted_img, mask = erase_black(grayscale_cam, img)

  return img_arr, mask

In [ ]:
pad = 'reflection' # 'zero'
OPT_OVER = 'net'
OPTIMIZER = 'adam'

In [ ]:
len(mask_list)

6478

# Load mask

In [ ]:
%cd '/content/pytorch-grad-cam/'
from pytorch_grad_cam import EigenCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, scale_cam_image
from pytorch_grad_cam.utils.model_targets import FasterRCNNBoxScoreTarget
from pytorch_grad_cam.utils.reshape_transforms import fasterrcnn_reshape_transform

## For Loop for Selective Inpainting

Play around with the num_iter, the LR, and the network being used. The ResNet does about an image a minute. Best idea is to split up names_augmented.txt into 8 files and run several colab notebooks at the same time.

Make sure to change the path in savefig to be a path in your google drive

In [ ]:
for name in mask_list:
  img_path = "/content/original_data/JPEGImages/trainval/"+name

  img_np, img_mask_np = get_image_mask(img_path)

  img_var = np_to_torch(img_np).type(dtype)
  mask_var = np_to_torch(img_mask_np).type(dtype)

  INPUT = 'noise'
  input_depth = 32
  LR = 0.001 
  num_iter = 4001
  param_noise = False
  show_every = 50
  figsize = 5
  reg_noise_std = 0.03
  
  net = ResNet(input_depth, img_np.shape[0], 8, 32, need_sigmoid=True, act_fun='LeakyReLU')

  net = net.type(dtype)
  net_input = get_noise(input_depth, INPUT, img_np.shape[1:]).type(dtype)

  # Compute number of parameters
  s  = sum(np.prod(list(p.size())) for p in net.parameters())
  print ('Number of params: %d' % s)

  # Loss
  mse = torch.nn.MSELoss().type(dtype)

  img_var = np_to_torch(img_np).type(dtype)
  mask_var = np_to_torch(img_mask_np).type(dtype)
  mask_var = mask_var[:, :, :, None]
  mask_var = mask_var.repeat(1,1,1,3)

  i = 0
  def closure():
      
      global i
      
      if param_noise:
          for n in [x for x in net.parameters() if len(x.size()) == 4]:
              n = n + n.detach().clone().normal_() * n.std() / 50
      
      net_input = net_input_saved
      if reg_noise_std > 0:
          net_input = net_input_saved + (noise.normal_() * reg_noise_std)
          
      out = net(net_input)
      total_loss = mse(out * mask_var, img_var * mask_var)
      total_loss.backward()
          
      print ('Iteration %05d    Loss %f' % (i, total_loss.item()), '\r', end='')
      
      i += 1

      return total_loss

  net_input_saved = net_input.detach().clone()
  noise = net_input.detach().clone()

  p = get_params(OPT_OVER, net, net_input)
  optimize(OPTIMIZER, p, closure, LR, num_iter)

  out_np = torch_to_np(net(net_input))
  plt.clf()
  plt.imshow(out_np)
  plt.savefig('/content/drive/MyDrive/VLR_Project/data/PASCAL2012/PASCAL-INPAINTED/InpaintedFasterRCNN/'+img_path.split("/")[-1].split(".")[0]+'.jpg')
  print("saved image inpainted: ", img_path.split("/")[-1].split(".")[0])
  # plot_image_grid([out_np], factor=5);